In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

# Volume Estimation and Uncertainty Analysis

### 2D Volume Estimation

In petrophysics, 2D volume estimation is used to calculate the volume of hydrocarbons in a reservoir using two-dimensional data, typically from seismic surveys or well logs. Here's a simplified process:

1. **Data Collection**: Gather 2D seismic data and well log data. Seismic data provides a cross-sectional view of the subsurface, while well logs give detailed information about the rock and fluid properties at specific locations.

2. **Structural Mapping**: Create structural maps of the reservoir. This involves interpreting seismic data to delineate the boundaries and structure of the reservoir.

3. **Reservoir Zoning**: Divide the reservoir into zones based on geological and petrophysical properties. Each zone may have different characteristics affecting hydrocarbon volume.

4. **Property Assignment**: Assign petrophysical properties (e.g., porosity, water saturation) to each zone. These properties are derived from well logs and core samples.

5. **Volume Calculation**: Calculate the volume of each zone using the formula:
   $$
   V = A \times h \times \phi \times (1 - S_w)
   $$
   where:
   - \( V \) is the hydrocarbon volume.
   - \( A \) is the area of the zone.
   - \( h \) is the thickness of the zone.
   - \( \phi \) is the porosity.
   - \( S_w \) is the water saturation.

### Uncertainty Analysis

Uncertainty analysis is crucial because it helps quantify the confidence in the volume estimates. Here are the key steps:

1. **Identify Uncertainties**: Determine the sources of uncertainty, such as data quality, interpretation errors, and variability in petrophysical properties.

2. **Quantify Uncertainties**: Use statistical methods to quantify the uncertainties. This can involve:
   - **Monte Carlo Simulation**: Generate multiple realizations of the reservoir model by varying the input parameters within their uncertainty ranges.
   - **Sensitivity Analysis**: Assess how changes in each parameter affect the volume estimate.

3. **Probabilistic Volume Estimation**: Combine the results from the simulations to create a probabilistic distribution of the hydrocarbon volume. This provides a range of possible volumes with associated probabilities.

4. **Risk Assessment**: Evaluate the risk associated with different volume estimates. This helps in making informed decisions about reservoir development and management.

quick_pp estimates the volume by considering the distribution of each parameter. Smaller range and smaller standard deviation indicates less uncertainty and vice versa applies.

Monte Carlo simulation is implemented ...

In [ ]:
import pandas as pd

from quick_pp.objects import Project

# Load well from saved file
project = "MOCK_carbonate"
project_path = rf"data\04_project\{project}.qppp"
project = Project().load(project_path)
project.get_well_names()

In [39]:
df = project.get_all_data()

### Fluid Contact Analysis

In [ ]:
import pprint
from quick_pp.plotter.plotter import stick_plot, generate_well_config, generate_zone_config

zone_config = generate_zone_config()

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(zone_config)

In [ ]:
well_names = project.get_well_names()
well_config = generate_well_config(well_names)

pp.pprint(well_config)

In [42]:
from quick_pp.plotter.plotter import update_well_config

WATER_CONTACT = {
    'HW-3': 8559,
    'HW-4': 8559,
    'HW-5': 8559,
    'HW-6': 8559,
    'HW-7': 8545,
    'HW-8': 8545,
    'HW-9': 8567,
    'HW-10': 8567,
    'HW-24': 8559,
    'HW-25': 8559,
    'HW-26': 8559,
    'HW-27': 8567,
    'HW-28': 8567,
    'HW-29': 8567,
    'HW-30': 8567,
    'HW-31': 8545,
    'HW-32': 8545,
}

for well in well_names:
    fwl = WATER_CONTACT[well]
    fluid_dict = {
        'OUT': 7929,
        'OWC': fwl,
        'ODT': fwl,
        'WUT': fwl,
    }
    well_config = update_well_config(well_config, well, "ALL", fluid_dict)

In [ ]:
# 9, 27, 10, 30, 28, 29, 8, 31, 32, 7, 3, 24, 25, 6, 26, 4, 5
sorting_dict = { 
    'HW-10': 3,
    'HW-24': 12,
    'HW-25': 13,
    'HW-26': 15,
    'HW-27': 2,
    'HW-28': 5,
    'HW-29': 6,
    'HW-3': 11,
    'HW-30': 4,
    'HW-31': 8,
    'HW-32': 9,
    'HW-4': 16,
    'HW-5': 17,
    'HW-6': 14,
    'HW-7': 10,
    'HW-8': 7,
    'HW-9': 1
}

for well, sorting in sorting_dict.items():
    well_config = update_well_config(well_config, well, sorting=sorting)

pp.pprint(well_config)

In [ ]:
stick_plot(df, well_config)

### Statistical Summary

In [ ]:
from quick_pp.qaqc import quick_compare
from quick_pp.utils import zone_flagging

# df = project.get_all_data()

df = df[~(df.WELL_NAME == 'HW-3')]
df['VSHALE'] = df['VCLW']
df['SWT'] = df['SWT'].where(df['SWT'] <= 1, 1)
df = zone_flagging(df)
compare_df, fig = quick_compare(df, return_fig=True)

In [ ]:
from quick_pp.qaqc import extract_quick_stats

extract_quick_stats(compare_df, flag='pay')

### Volume Estimation

In [ ]:
from quick_pp.ressum import mc_volumetric_method

area_bound = (500, 2500, 1560, 400)
thickness_bound = (0, 216, 89, 74)
porosity_bound = (.066, .261, .177, .05)
water_saturation_bound = (.228, 1, .732, .297)
volume_factor_bound = (1.2, 1.4)

n_try = 10000
reserves = mc_volumetric_method(
    area_bound=area_bound,
    thickness_bound=thickness_bound,
    porosity_bound=porosity_bound,
    water_saturation_bound=water_saturation_bound,
    volume_factor_bound=volume_factor_bound,
    n_try=n_try, percentile=[10, 50, 90]
)

### Sensitivity Analysis

In [ ]:
from quick_pp.ressum import sensitivity_analysis

sensitivity_analysis(
    area_bound=area_bound,
    thickness_bound=thickness_bound,
    porosity_bound=porosity_bound,
    water_saturation_bound=water_saturation_bound,
    volume_factor_bound=volume_factor_bound
)